In [4]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch
device = "cuda:0"

In [5]:
load_model_name = "LDCC/LDCC-SOLAR-10.7B" # 학습된 모델 불러오기 가능
load_tokenizer_name = "LDCC/LDCC-SOLAR-10.7B"

models = AutoModelForCausalLM.from_pretrained(load_model_name).to(device)
tokenizers = AutoTokenizer.from_pretrained(load_tokenizer_name)

In [24]:
input_text = '면진구조에 대해 설명하시오.'
input_ids = tokenizers.encode(
            input_text, padding="max_length", max_length="128", return_tensors="pt", add_special_tokens=False
        )
print(input_ids)

TypeError: 'str' object cannot be interpreted as an integer

In [23]:
output_sequences = models.generate(
        input_ids=input_ids.to("cuda"),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

tokenizers.decode(output_sequences[0])

'면진구조에 대해 설명하시오.\n[A]: ②\n\n[Q]: 19세기의 미국과는\n[A]:\n산업</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s